In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "Alekesei Brisikhin Spark RDD app") 

sc = SparkContext(conf=conf)

In [3]:
sc

<SparkContext master=yarn appName=Alekesei Brisikhin Spark RDD app>

---

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


In [5]:
film_id_for_calculations = '22'

---

In [6]:
# user id | item id | rating | timestamp

In [7]:
data = (sc.textFile('/labs/laba01/ml-100k/u.data')
        .map(lambda x: x.split('\t')[1:3])
        .cache())

In [8]:
res_for_one = (data
               .filter(lambda x: x[0] == film_id_for_calculations)
               .map(lambda x: (x[1], 1))
               .reduceByKey(lambda x, y: x + y)
               .collect())

res_for_one = sorted(res_for_one, key=lambda x: x[0])
res_for_one

[('1', 5), ('2', 14), ('3', 46), ('4', 98), ('5', 134)]

In [9]:
res_for_all = (data
               .map(lambda x: (x[1], 1))
               .reduceByKey(lambda x, y: x + y)
               .collect())

res_for_all = sorted(res_for_all, key=lambda x: x[0])
res_for_all

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [11]:
result = {'hist_film': [r[1] for r in res_for_one], 
          'hist_all': [r[1] for r in res_for_all]}

result

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [12]:
import json

with open('./lab01.json', 'w') as f:
    json.dump(result, f)

In [15]:
!ls -l | grep json

-rw-rw-r-- 1 alexey.borisikhin alexey.borisikhin   83 Mar  2 21:49 lab01.json


In [17]:
!cat ./lab01.json

{"hist_film": [5, 14, 46, 98, 134], "hist_all": [6110, 11370, 27145, 34174, 21201]}